In [3]:
from IPython.display import display
from utils import close_notebook, AppWidgets

In [4]:
# Widgets
app_widgets = AppWidgets("Create_sample", "config.json")

# Create a sample object in openBIS

## Select material

In [5]:
app_widgets.material_selection_radio_button.observe(app_widgets.select_material_radio_change, names='value')
display(app_widgets.material_selection_radio_button)

RadioButtons(layout=Layout(width='300px'), options=('No material', 'Crystal', 'Wafer substrate', '2D-layer mat…

## Sample out

In [ ]:
app_widgets.create_button.on_click(app_widgets.create_sample_action)
app_widgets.quit_button.on_click(close_notebook)
display(app_widgets.sample_out_name_textbox, app_widgets.bottom_buttons_hbox)
display(app_widgets.increase_buttons_size)